<a href="https://colab.research.google.com/github/thimotyb/spark-notebooks/blob/main/mlflow_quickstart_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Compare runs, choose a model, and deploy it to a REST API
- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLflow UI
- Choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform


https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html



In [ ]:
!pip install tensorflow
!pip install hyperopt

In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip list

Package                            Version
---------------------------------- -----------
absl-py                            2.1.0
alembic                            1.13.3
asttokens                          2.4.1
astunparse                         1.6.3
blinker                            1.8.2
boltons                            23.0.0
brotlipy                           0.7.0
cachetools                         5.5.0
certifi                            2024.8.30
cffi                               1.15.1
charset-normalizer                 2.0.4
click                              8.1.7
cloudpickle                        3.1.0
comm                               0.2.2
conda                              23.9.0
conda-content-trust                0.2.0
conda-libmamba-solver              23.9.1
conda-package-handling             2.2.0
conda_package_streaming            0.9.0
contourpy                          1.3.0
cryptography                       41.0.3
cycler                             0.12

In [4]:
from tensorflow import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [5]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [6]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [7]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [8]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result



In [9]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [10]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2024/10/24 11:38:10 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 25s 562ms/step - loss: 32.5727 - root_mean_squared_error: 5.0447
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25.8879 - root_mean_squared_error: 4.3208
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.0674 - root_mean_squared_error: 4.3208
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19.4333 - root_mean_squared_error: 4.3208
36/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5532 - root_mean_squared_error: 2.2061 
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.8270 - root_mean_squared_error: 2.0456 - val_loss: 0.6717 - val_root_mean_squared_error: 0.8196

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 14s 329ms/step - loss: 0.8974 - root_mean_squared_error: 0.9473
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8613 - root_mean_squared_error: 0.9278
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7361 - root_mean_squared_error: 0.8295   
 4/46 ━━━━━━━━━━━━━━━━━━━━

2024/10/24 11:38:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run bold-worm-79 at: http://127.0.0.1:8080/#/experiments/363613253566840610/runs/7116ea82e557453090ac73e6fddf694a.

2024/10/24 11:38:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/363613253566840610.



Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 21s 488ms/step - loss: 34.6092 - root_mean_squared_error: 5.8830
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.4734 - root_mean_squared_error: 5.7848
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.2998 - root_mean_squared_error: 5.4950
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.9736 - root_mean_squared_error: 5.4950
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.4842 - root_mean_squared_error: 5.1384   
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.7925 - root_mean_squared_error: 3.6261
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 12.8454 - root_mean_squared_error: 3.4894
39/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.4564 - root_mean_squared_error: 3.5781
40/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.2962 - root_mean_squared_error: 3.5326
41/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.1415 - root_mean_squared_error: 3.4894
42/46 ━━━━━━━━━━━━━━━━━━

2024/10/24 11:38:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run aged-fox-632 at: http://127.0.0.1:8080/#/experiments/363613253566840610/runs/0064fb215e8f4443b8d092fda4885eea.

2024/10/24 11:38:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/363613253566840610.



...

Epoch 1/3                                                                     

 1/46 ━━━━━━━━━━━━━━━━━━━━ 22s 491ms/step - loss: 35.8419 - root_mean_squared_error: 5.9868
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 35.1188 - root_mean_squared_error: 5.9258
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.3556 - root_mean_squared_error: 5.8610   
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.5271 - root_mean_squared_error: 5.8516   
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.7250 - root_mean_squared_error: 5.8516   
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.0756 - root_mean_squared_error: 5.6617
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.8663 - root_mean_squared_error: 5.6430
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.9695 - root_mean_squared_error: 5.6337
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.6579 - root_mean_squared_error: 5.6242
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.7628 - root_mean_squared_error: 5.6337
28/46 ━━━━━━━━━━━━

2024/10/24 11:38:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run flawless-bat-518 at: http://127.0.0.1:8080/#/experiments/363613253566840610/runs/27a47c913b854c1897e076ea7ed7c9cf.

2024/10/24 11:38:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/363613253566840610.



100%|██████████| 8/8 [00:45<00:00,  5.67s/trial, best loss: 0.733542263507843]


2024/10/24 11:38:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-kite-762 at: http://127.0.0.1:8080/#/experiments/363613253566840610/runs/0355a9ec4ea848cbad306cf6a47911ca.
2024/10/24 11:38:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/363613253566840610.


Best parameters: {'lr': 0.09418237371588713, 'momentum': 0.41158741240278984}
Best eval rmse: 0.733542263507843


- Open MLFlow UI and Inspect Rsults at http://localhost:8080
- Chart View
- Register the Model

- Serve the Model Locally: (in terminal)
curl https://pyenv.run | bash
pip install virtualenv
export MLFLOW_TRACKING_URI=http://localhost:8080
mlflow models serve -m "models:/wine-quality/1" --port 5002

- To test the model:
curl -d '{"dataframe_split": {
"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],
"data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' \
-H 'Content-Type: application/json' -X POST localhost:5002/invocations



In [ ]:
- Build a model:
mlflow models build-docker --model-uri "models:/wine-quality/1" --name "qs_mlops"
docker run -p 5002:8080 qs_mlops
curl -d '{"dataframe_split": {"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], "data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' -H 'Content-Type: application/json' -X POST localhost:5002/invocations

